## helioMOPS (MOPS-TNG)

An effort to kill the tracklet, make our lives easier, and a lot of people happier. 

### Background

The current implementation of the LSST Moving Object Processing System requires the LSST cadence to deliver at least two visits a night to a single field. Two visits to a single field allows for the creation of tracklets: sky-plane motion vectors in the reference frame of the observer that constrain the direction and velocity of a potential moving object. At least two visits a night followed by another set about three days later is necessary for LSST to deliver its Solar System science data products, but two visits a nights is not required by the other three science drivers. It is of interest to find a Solar System linking algorithm that delivers the same if not better performance as LSST MOPS but that does not limit LSST's cadence to be driven by Solar System science.

### Proposed Algorithm

Considering the simplest case: a DIASource detection will at the very least provide us with information as to the location of the detection, the time, the magnitude (in a specific filter), and the associated signal to noise ratio. 

$$ \text{DIASource} : [(t_i, \alpha_i, \delta_i), m_{f, i}, snr_{f, i}] $$

In addition to certain DIASource parameters, we know the location of the observer (the telescope) and the location of the Sun. These can be expressed in a variety of different coordinate systems: equatorial, ecliptic or cartesian. Transformations between coordinate systems are trivial and have been implemented in a variety of well-tested and robust software packages. To maintain generality we will proceed with cartesian coordinates. We assume the origin is at the location of the Sun with the reference axes are tied to J2000 epoch and the ecliptic:

$$ \vec{U_{obs}} (t_i) = (x_{obs}(t_i), y_{obs}(t_i), z_{obs}(t_i)) $$

The $\text{DIASource}$ object and the $\vec{U_{obs}}$ vector summarize the extent of our knowledge in the simplest case. 

In order to link any DIASource detections we need solve for either $r_{as}$, the asteroid to sun distance, or, $r_{ao}$, the asteroid to observer distance. Note that in this example we refer to an asteroid: this method should be applicable to a variety of small body populations. 

It is more difficult to guess $r_{ao}$ accurately since the Earth's motion needs to be taken into account. However, we can proceed by guessing $r_{as}$. So lets assume we have an estimate of $r_{as}$. Note that we can couple the the heliocentric distance with the estimated velocity that the asteroid travels with a simple Keplerian model. 

If we assume some reasonable heliocentric distance we can transform equatorial coordinates $(\alpha_i, \delta_i)$ to cartesian coordinates $(x_i, y_i, z_i)$. From there we can perform a procedure similar to the work of Bernstein & Khushalani and transform to tangent plane coordinates from the reference point of the Sun. These tangent plane coordinates take the form $(\theta_x, \theta_y)$. 

In the ideal case, each individial asteroid will generate many different DIASources. Within the span of a few days, or in other words, within the span of a few visits to the same or nearby fields we expect any small body to generate DIASources. If for any given field we make a guess at some reasonable heliocentric distance we can create a set of tangent plane projections. 

In this tangent plane projection in the frame of the Sun we can phase-fold nearby field visits assuming some velocity (in principle this can be done at the per field level or at the per DIASource level). If our estimate of the velocity and heliocentric distance was correct we would anticipate that in the phase-folded image a true object would appear as a clustered set of points. 

In the likely event that we guessed the distance and velocity incorrectly for a subset of DIASources tied to a population of asteroids we would expect to see some linear features in the phase-folded image. Using a Hough transform these features can be extracted and we can then calculate the most probable heliocentric distance and establish potential linkages.

(To test: if we assume some rotational direction about the Sun, we could check if we overestimated or underestimated the distance by looking at the direction that these trails take in the phase-folded image.)


### Proposed Experimental Implementation

1) Grab the latest MPCORB.dat

2) Select a 60 degree population radius and generate ~30 days of ephemeris at 1 day intervals

3) Group detections into simulated LSST fields (simulated as in: within 3.75 degrees of some reference center point)

4) Take a single set of fields and make a guess at the heliocentric distance

5) Perform equatorial - cartesian - tangent plane transform

6) Phase-fold nearby images in tangent plane coordinates assumimg some velocity (~1 deg / day to start)

7) Look for any clustered points (by eye at first)

8) Peform Hough transform to identify any linear features

9) Repeat 4-8 as necessary